# Muehle Game Definition
This file contains the definiton of the game Muehle.

The starting state of Muehle is defined as following. It contains multiple tuples:

1. The first tuple contains the number of stones white and black are still allowed to place.
2. The second tuple contains the state of the board. Every nine-tuple represents a ring of the possible positions of a stone starting at the top left corner continuing clock-wise. The positions can ether by empty `' '`, occupied by white `'w'` or occupied by black `'b'`.

In [1]:
s = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))
s2 = ((0, 0), (
    ('w', 'w', ' ', 'b', 'w', 'w', 'w', 'w'),
    ('b', 'w', ' ', 'b', 'w', ' ', 'b', 'w'),
    ('b', 'b', 'b', 'b', 'w', ' ', 'b', 'w'),
))
s3 = ((7, 7), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))

## Calculate player phase

In [2]:
# Calculates wether or not a player still has stones which he has not placed yet
#   s = current state
#   p = player
def canAddStone(s, p):
    # Extract the count of the stones for black and white
    ((cw, cb), _) = s
    # Return wether or not the given player has at least one stone
    return cw >= 1 if p == 'w' else cb >= 1

In [3]:
# Counts how many stones the given player has left on the board
def countStones(s, p):
    # Extract the board from the state
    (_, board) = s
    # Count how many times player occurs on the board
    return [cell for ring in board for cell in ring].count(p)

In [4]:
# Calculates wether or not the player is allowed to jump with his stones,
# instead of just moving them
def canJump(s, p):
    return countStones(s, p) >= 3

In [5]:
def playerPhase(s, p):
    # If the player has still stones left to place, he is still in phase 1
    if canAddStone(s, p):
        return 1
    # If the player is allowed to jump with his stones, he is in the last phase, phase 3
    elif canJump(s, p):
        return 3
    # Else he is in phase 2, where he can only move his stones
    else:
        return 2

## Calculate stone additions

In [6]:
# Returns a set of tuples containing all coordinates of the cells owned by p
# Set of Tuples(ring, cell)
def findCellsOf(board, p):
    # Iterate over all cells and select only empty cells
    return {(r, c) for r in range(0, 3) for c in range(0, 8) if board[r][c] == p}

In [7]:
# Returns a set of tuples containing all coordinates of the empty cells
# Set of Tuples(ring, cell)
def findEmptyCells(s):
    # Extract the board from the state
    (_, board) = s
    return findCellsOf(board, ' ')

In [8]:
# Returns the coordinates of all Muehlen the given player has
# Set of Frozensets of Tuples(ring, cell)
def findMuehlen(board, p):
    # Calculate all Muehlen on the rings
    return {
        frozenset((r, (c+o)%8) for o in range(0, 3))
        # Iterate over all rings
        for r in range(0, 3)
        # Iterate over all corners
        for c in range(0, 8, 2)
        # All 3 following cells starting at the given corner have to belong to the player
        if all(
            cell == p
            # Iterate over all 3 cells of the given side (c) by rotating them to the top left corner
            # and then checking the first 3 cells
            for cell in (board[r][c:] + board[r][:c])[0:3]
        )
    # Calculate all Muehlen crossing the rings
    } | {
        frozenset((r, c) for r in range(0, 3))
        # Iterate over cells in the middle of a side
        for c in range(1, 8, 2)
        # All 3 cells in the middle of a given side have to belong to the player
        if all(
            board[r][c] == p
            # Iterate over all 3 rings
            for r in range(0, 3)
        )
    }

In [9]:
def place(board, coordinates, player):
    (r, c) = coordinates
    return tuple(
        tuple(
            player if (c == ic) and (r == ir) else board[ir][ic]
            for ic in range(0, 8)
        ) for ir in range(0, 3)
    )

In [10]:
# Calculates the opponent of the given player
def opponent(p):
    return 'b' if p == 'w' else 'w'

In [14]:
def nextStatesAddStones(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current Muehlen the player has
    muehlen = findMuehlen(board, p)

    boards = {
        place(board, (r, c), p)
        for (r, c) in findEmptyCells(s)
    }
    boardsWithMuehlen = {
        board
        for board in boards
        if len(findMuehlen(board, p) - muehlen) > 0 
    }
    boards -= boardsWithMuehlen

    boards |= {
        place(board, (r, c), ' ')
        for board in boardsWithMuehlen
        for (r, c) in findCellsOf(board, opponent(p))
    }

    # Remove one stone from the players stache
    (cw, cb) = (cw-1, cb) if p == 'w' else (cw, cb-1)

    # Return all possible states
    return { ((cw, cb), board) for board in boards }
nextStatesAddStones(s3, 'w')

{((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', ' ', ' ', 'b', ' ', 'w'))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', ' ', ' ', 'b', 'w', ' '))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', ' ', 'w', 'b', ' ', ' '))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'w', ' ', 'b', ' ', ' '))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', ' ', 'w', ' ', ' ', 'b', ' ', ' '))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   (' ', 'w', ' ', ' ', ' ', 'b', ' ', ' '))),
 ((6, 7),
  (('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
   (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
   ('w', ' ', ' ',